# Imports
This notebook uses data available to FIREMAN members on [LUT sharepoint](https://lut.sharepoint.com/:f:/r/sites/o365fireman/Shared%20Documents/Colab_PowerElectronicConverter/PEC_datasets/New%20Data?csf=1&web=1&e=JINTw3)

In [1]:
import sys
import pandas as pd
# to save results to data directory
module_path = '..'
if module_path not in sys.path:
    sys.path.insert(0, module_path)
# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

In [2]:
import scipy.io
import src.common as common

# increase displayed columns in jupyter notebook
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 300)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

# temporarily remove deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

output_dir = 'pec_output'

# Dataset

**identifiers**

In [3]:
column_types = common.json_load("#datasets/PEC/column_types.json")

identifier = column_types["identifier"]
target = column_types["target"]
datasets = column_types["datasets"]
feature_ids = column_types["feature_ids"]

**agregate data from matlab files**

In [4]:
data_df_final = pd.DataFrame(columns = list(feature_ids.keys()).extend(["fault_id", "fault"]))

for dataset in datasets.keys():
    data_raw_dict = {}
    data_raw = scipy.io.loadmat("#datasets/PEC/" + dataset + "/in.mat")
    in_len = len(data_raw["in"])
    for feature in feature_ids.keys():
        data_raw_dict.update({feature: data_raw[feature_ids[feature][0]][:in_len,feature_ids[feature][1]]})
    data_raw_dict.update({"label": [dataset]*in_len})
    data_df = pd.DataFrame(data_raw_dict)
    data_df["fault"] = 0
    if len(datasets[dataset]["failure"][0])==1:
        data_df.loc[datasets[dataset]["failure"][0][0]:, "fault"] = 1
    if len(datasets[dataset]["failure"][0])==2:
        data_df.loc[datasets[dataset]["failure"][0][0]:datasets[dataset]["failure"][0][1], "fault"] = 1
    data_df.drop(data_df.index[datasets[dataset]["drop"][0][0]:datasets[dataset]["drop"][0][1]], inplace=True)
    data_df_final = pd.concat([data_df_final, data_df], ignore_index=True)
data_df_final[identifier] = data_df_final.index

/Users/abeattie/miniconda/envs/masters-thesis/lib/python3.10/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/Users/abeattie/miniconda/envs/masters-thesis/lib/python3.10/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/Users/abeattie/miniconda/envs/masters-thesis/lib/python3.10/site-packages/scipy/io/matlab/mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)
/Users/abeattie/miniconda/envs/masters-thesis/lib/python3.10/sit

**save to pickle for further use**

In [5]:
data_df_final.to_pickle("#datasets/PEC/dataset.pkl")
data_df_final.to_pickle("../masters-thesis-graphing/_data/PEC/dataset.pkl")

## Simple display of the dataset

In [6]:
data_df_final.head()

,f_c,P,m_d,m_q,theta,P_ref,V_DC,V_phaseA,V_phaseB,V_phaseC,I_phaseA,I_phaseB,I_phaseC,label,fault,
0,50.0,2499.997221,311.0,0.0,312.486512,2500.0,800.0,2.487576,-270.569073,268.081497,-0.392875,-4.447507,4.840382,LL_Fault,0,0
1,50.0,2499.997250,311.0,0.0,312.502220,2500.0,800.0,7.372088,-272.944264,265.572176,-0.308598,-4.494417,4.803015,LL_Fault,0,1
2,50.0,2499.997280,311.0,0.0,312.517928,2500.0,800.0,12.254782,-275.252112,262.997330,-0.224245,-4.540218,4.764463,LL_Fault,0,2
3,50.0,2499.997310,311.0,0.0,312.533636,2500.0,800.0,17.134452,-277.492044,260.357593,-0.139837,-4.584899,4.724735,LL_Fault,0,3
4,50.0,2499.997341,311.0,0.0,312.549344,2500.0,800.0,22.009894,-279.663510,257.653616,-0.055394,-4.628448,4.683842,LL_Fault,0,4


In [7]:
data_df_final.plot(subplots=True, figsize=(16,15));

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
